# Lab 2: Building a supervised machine learning model

Your objectives for this lab are to:
* practice basic preprocessing steps to set up a supervised learning task,
* learn how to explore a dataset to understand its structure, variables, and missingness, and
* implement `KNeighborsClassifier` and adjust its hyperparameters.

First, let's make the necessary imports for today with the code below.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

Now load the Breast Cancer dataset, which is provided by `sklearn`.

In [2]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

# Part 1: Preprocessing and EDA

A widely used library for data wrangling is `pandas` (`pd`for short), which you imported above. To work with data using `pandas`, we first need convert the `cancer` dataset into a `DataFrame`. To do this, run the code below.

Notice how we also specify the target variable — the variable we'll try to to predict with `KNeighborsClassifier` later on in this notebook.

In [3]:
df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
df['target'] = cancer.target

Now check the shape of `df` and inspect the column names.

In [4]:
print("Our dataframe has", df.shape[0], "rows and", df.shape[1], "columns.")
print("The column names are:", df.columns)

Our dataframe has 569 rows and 31 columns.
The column names are: Index(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error', 'fractal dimension error',
       'worst radius', 'worst texture', 'worst perimeter', 'worst area',
       'worst smoothness', 'worst compactness', 'worst concavity',
       'worst concave points', 'worst symmetry', 'worst fractal dimension',
       'target'],
      dtype='object')


Do you know what those column names are referring to?

If not, a good first step for working with the data would be to familiarize
yourself with the to source of the data, how it was collected, and what each variable is measuring. Here's where you can find that information: https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic

## Question 1
No let's start exploring the data. Use `describe()` to get some descriptive statistics.

Then print the different values of the target column, and how many occurrences there are of each value. What does each value of the target variable represent?

In [6]:
df.describe()


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,...,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946,0.627417
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,...,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061,0.483918
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,...,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040,0.000000
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,...,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460,0.000000
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,...,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040,1.000000
75%,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,...,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080,1.000000
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,...,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500,1.000000


In [5]:

print(df['target'])

0      0
1      0
2      0
3      0
4      0
      ..
564    0
565    0
566    0
567    0
568    1
Name: target, Length: 569, dtype: int32


## Question 2

In the real world, the datasets you work with often have missing values. This usually isn't a problem if there are very few missing values, or if the values are missing completely at random. But, if there are lots of missing values and/or there's a pattern to the missingness (e.g., there's data missing on all patients above 50 years-old, there's data missing on all female patients, etc.), then missing values could indicate that there's a bias in the dataset. So, it's important to check for missing values.

Use `isnull().sum()` to count the number of missing values in each column.

(*Hint: there should be zero missing values in this dataset*)


In [7]:

print(df.isnull().sum())

mean radius                0
mean texture               0
mean perimeter             0
mean area                  0
mean smoothness            0
mean compactness           0
mean concavity             0
mean concave points        0
mean symmetry              0
mean fractal dimension     0
radius error               0
texture error              0
perimeter error            0
area error                 0
smoothness error           0
compactness error          0
concavity error            0
concave points error       0
symmetry error             0
fractal dimension error    0
worst radius               0
worst texture              0
worst perimeter            0
worst area                 0
worst smoothness           0
worst compactness          0
worst concavity            0
worst concave points       0
worst symmetry             0
worst fractal dimension    0
target                     0
dtype: int64


## Question 3

Which attributes correlate with the target variable? Do any of the attributes correlate with one another?

Produce a correlation matrix to check and get an inital sense of relationships between variables.

In [8]:

df.corr()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
mean radius,1.000000,0.323782,0.997855,0.987357,0.170581,0.506124,0.676764,0.822529,0.147741,-0.311631,...,0.297008,0.965137,0.941082,0.119616,0.413463,0.526911,0.744214,0.163953,0.007066,-0.730029
mean texture,0.323782,1.000000,0.329533,0.321086,-0.023389,0.236702,0.302418,0.293464,0.071401,-0.076437,...,0.912045,0.358040,0.343546,0.077503,0.277830,0.301025,0.295316,0.105008,0.119205,-0.415185
mean perimeter,0.997855,0.329533,1.000000,0.986507,0.207278,0.556936,0.716136,0.850977,0.183027,-0.261477,...,0.303038,0.970387,0.941550,0.150549,0.455774,0.563879,0.771241,0.189115,0.051019,-0.742636
mean area,0.987357,0.321086,0.986507,1.000000,0.177028,0.498502,0.685983,0.823269,0.151293,-0.283110,...,0.287489,0.959120,0.959213,0.123523,0.390410,0.512606,0.722017,0.143570,0.003738,-0.708984
mean smoothness,0.170581,-0.023389,0.207278,0.177028,1.000000,0.659123,0.521984,0.553695,0.557775,0.584792,...,0.036072,0.238853,0.206718,0.805324,0.472468,0.434926,0.503053,0.394309,0.499316,-0.358560
mean compactness,0.506124,0.236702,0.556936,0.498502,0.659123,1.000000,0.883121,0.831135,0.602641,0.565369,...,0.248133,0.590210,0.509604,0.565541,0.865809,0.816275,0.815573,0.510223,0.687382,-0.596534
mean concavity,0.676764,0.302418,0.716136,0.685983,0.521984,0.883121,1.000000,0.921391,0.500667,0.336783,...,0.299879,0.729565,0.675987,0.448822,0.754968,0.884103,0.861323,0.409464,0.514930,-0.696360
mean concave points,0.822529,0.293464,0.850977,0.823269,0.553695,0.831135,0.921391,1.000000,0.462497,0.166917,...,0.292752,0.855923,0.809630,0.452753,0.667454,0.752399,0.910155,0.375744,0.368661,-0.776614
mean symmetry,0.147741,0.071401,0.183027,0.151293,0.557775,0.602641,0.500667,0.462497,1.000000,0.479921,...,0.090651,0.219169,0.177193,0.426675,0.473200,0.433721,0.430297,0.699826,0.438413,-0.330499
mean fractal dimension,-0.311631,-0.076437,-0.261477,-0.283110,0.584792,0.565369,0.336783,0.166917,0.479921,1.000000,...,-0.051269,-0.205151,-0.231854,0.504942,0.458798,0.346234,0.175325,0.334019,0.767297,0.012838


 ## Question 4

Which variables display the strongest correlation with the `target` variable?

 Select the final column of the correlation matrix (the `target` column), sort the column by absolute values, and display it.

the variable smoothness error seems to have the largest correlation with the target variable

In [12]:
df.corrwith(df['target'])


mean radius               -0.730029
mean texture              -0.415185
mean perimeter            -0.742636
mean area                 -0.708984
mean smoothness           -0.358560
mean compactness          -0.596534
mean concavity            -0.696360
mean concave points       -0.776614
mean symmetry             -0.330499
mean fractal dimension     0.012838
radius error              -0.567134
texture error              0.008303
perimeter error           -0.556141
area error                -0.548236
smoothness error           0.067016
compactness error         -0.292999
concavity error           -0.253730
concave points error      -0.408042
symmetry error             0.006522
fractal dimension error   -0.077972
worst radius              -0.776454
worst texture             -0.456903
worst perimeter           -0.782914
worst area                -0.733825
worst smoothness          -0.421465
worst compactness         -0.590998
worst concavity           -0.659610
worst concave points      -0

In [13]:

df['target'].sort_values(key=abs)

0      0
190    0
193    0
194    0
196    0
      ..
269    1
268    1
267    1
281    1
568    1
Name: target, Length: 569, dtype: int32

# Part 2: Modeling

Now that you've done some basic preprocessing and data exploration, let's implement a `KNeighborsClassifier` to predict which tumors are cancerous (where `target = 0`).

## Question 5

Assign the target column to `y`, and assign the dataframe with the target column removed yo `X`.

Create a test train split using `X` and `y`.


In [19]:

cleandf= df.drop(['target'], axis=1)
x=cleandf
y=df['target']
x_train, x_test, y_train, y_test = train_test_split(x,y)

## Question 6

Create a `KNeighborsClassifier`. Use the fit classifier to the training data. Then print both the training and test scores.

How accurately does the `KNeighborsClassifier` predict whether a tumor is cancerous vs. benign?

In [20]:
KNN = KNeighborsClassifier(n_neighbors=5)

KNN.fit(x_train,y_train)

print(KNN.score(x_train,y_train))
print(KNN.score(x_test,y_test))

0.9460093896713615
0.9230769230769231


this model predicts with a 92.3% accuracy

## Question 7
Create a loop that builds kNN classifiers with either distance or uniform weighting, with numbers of neighbors varying between 1 and 20. What is the best combination? Produce a list consisting of test accuracy, training accuracy, number of neighbors and weighting choice. The list should be sorted by test accuracy.

<hr>

<i>Hints: </i>
<br>
<i>
Create two dicts: training_accuracy and test_accuracy. Make two loops: an outer loop ranging over number of neighbors, and an inner loop ranging over weighting choice. Then build the model with the current options and create a key unique to that option (eg. by concatenating the two options). Then you can store the current results in the the training and test dicts, using the current key. When you're done with the loops, you print out the sorted results of the two dicts.</i>

In [21]:

training_accuracy = {}
test_accuracy = {}


for n_neighbors in range(1, 21):
    for weight in ["uniform", "distance"]:
        

        knn = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weight)
        knn.fit(x_train, y_train)
        

        train_acc = knn.score(x_train, y_train)
        test_acc = knn.score(x_test, y_test)
        
        key = f"{n_neighbors}_{weight}"
        
        training_accuracy[key] = train_acc
        test_accuracy[key] = test_acc

sorted_results = sorted(test_accuracy.items(), key=lambda x: x[1], reverse=True)

print("Test Accuracy | Train Accuracy | Neighbors | Weighting")
for key, test_acc in sorted_results:
    train_acc = training_accuracy[key]
    n_neighbors, weight = key.split("_")
    print(f"{test_acc:.4f} | {train_acc:.4f} | {n_neighbors} | {weight}")




Test Accuracy | Train Accuracy | Neighbors | Weighting
0.9301 | 1.0000 | 5 | distance
0.9231 | 0.9695 | 2 | uniform
0.9231 | 0.9460 | 5 | uniform
0.9231 | 0.9437 | 6 | uniform
0.9231 | 1.0000 | 6 | distance
0.9161 | 1.0000 | 1 | uniform
0.9161 | 1.0000 | 1 | distance
0.9161 | 1.0000 | 2 | distance
0.9161 | 0.9577 | 3 | uniform
0.9161 | 1.0000 | 3 | distance
0.9161 | 1.0000 | 4 | distance
0.9161 | 0.9437 | 7 | uniform
0.9161 | 1.0000 | 7 | distance
0.9161 | 1.0000 | 8 | distance
0.9161 | 0.9343 | 17 | uniform
0.9161 | 0.9343 | 18 | uniform
0.9161 | 1.0000 | 18 | distance
0.9161 | 0.9343 | 19 | uniform
0.9161 | 1.0000 | 19 | distance
0.9161 | 0.9343 | 20 | uniform
0.9161 | 1.0000 | 20 | distance
0.9091 | 0.9507 | 4 | uniform
0.9091 | 0.9484 | 8 | uniform
0.9091 | 0.9390 | 9 | uniform
0.9091 | 1.0000 | 9 | distance
0.9091 | 0.9390 | 10 | uniform
0.9091 | 1.0000 | 10 | distance
0.9091 | 0.9437 | 11 | uniform
0.9091 | 1.0000 | 11 | distance
0.9091 | 0.9437 | 12 | uniform
0.9091 | 1.0000 | 1